In [21]:
import requests
import os
import logging
import sys
import json
logging.basicConfig(stream=sys.stderr, level=logging.INFO)

In [17]:
CARTO_URL = 'https://{}.carto.com/api/v2/sql'
CARTO_USER = ''
CARTO_KEY = ''
STRICT = True

In [30]:
def getTables(user=CARTO_USER, key=CARTO_KEY, f='csv'):
    '''Get the list of tables'''
    r = get('SELECT * FROM CDB_UserTables()', f=f)
    if f == 'csv':
        return r.text.split("\r\n")[1:-1]
    return r

def sendSql(sql, user=CARTO_USER, key=CARTO_KEY, f='', post=True):
    '''Send arbitrary sql and return response object or False'''
    url = CARTO_URL.format(user)
    payload = {
        'api_key': key,
        'q': sql,
    }
    if len(f):
        payload['format'] = f
    logging.debug((url, payload))
    if post:
        r = requests.post(url, json=payload)
    else:
        r = requests.get(url, params=payload)
    if not r.ok:
        logging.error(r.text)
        if STRICT:
            raise Exception(r.text)
        return False
    return r

def get(sql, user=CARTO_USER, key=CARTO_KEY, f=''):
    '''Send arbitrary sql and return response object or False'''
    return sendSql(sql, user, key, f, False)

def post(sql, user=CARTO_USER, key=CARTO_KEY, f=''):
    '''Send arbitrary sql and return response object or False'''
    return sendSql(sql, user, key, f)

def dropTable(table, user=CARTO_USER, key=CARTO_KEY):
    '''Delete table'''
    sql = 'DROP TABLE "{}"'.format(table)
    return post(sql)

In [33]:
tables = getTables()
# Load config options
try:
    table_info = json.load(open('/Users/nathansuberi/Documents/GitHub/ResourceWatchCode/Metadata Management/table_info.json', 'r'))
except Exception as e:
    logging.info(e)
    table_info = {}

{'%20soc_023_fragile_states_index': {'error': '* Problem fetching %20soc_023_fragile_states_index from Carto server *'},
 'cit_022_road_traffic_deaths_by_country': {'config_options': {'_incl': True,
   'country_code': False,
   'country_name': 'country',
   'id_cols': ['country', 'death_rate_per_100000'],
   'parse_date': True,
   'prefixes': ['estimated_number_of_road_traffic_deaths']}},
 'cit_025_urban_population': {'config_options': {'_incl': True,
   'country_code': 'country_code',
   'country_name': 'country_name',
   'id_cols': ['country_code',
    'country_name',
    'indicator_code',
    'indicator_name'],
   'parse_date': True,
   'prefixes': ['yr']}},
 'cit_028_percent_urban_slums_edit': {'config_options': {'_incl': True,
   'country_code': 'country_code',
   'country_name': 'country_name',
   'id_cols': ['country_code',
    'country_name',
    'indicator_code',
    'indicator_name'],
   'parse_date': True,
   'prefixes': ['yr']}},
 'cit_029_municipal_waste': {'config_options

In [34]:
matches = {}
for name in table_info:
    matches[name] = [table for table in tables if name in table]

In [35]:
matches

{'%20soc_023_fragile_states_index': [],
 'cit_022_road_traffic_deaths_by_country': ['cit_022_road_traffic_deaths_by_country',
  'cit_022_road_traffic_deaths_by_country_georefed__longform'],
 'cit_025_urban_population': ['cit_025_urban_population',
  'cit_025_urban_population_georefed__longform'],
 'cit_028_percent_urban_slums_edit': ['cit_028_percent_urban_slums_edit',
  'cit_028_percent_urban_slums_edit_georefed__longform'],
 'cit_029_municipal_waste': ['cit_029_municipal_waste',
  'cit_029_municipal_waste_georefed__longform'],
 'cli_007_greenhouse_gas_emissions_projections': ['cli_007_greenhouse_gas_emissions_projections',
  'cli_007_greenhouse_gas_emissions_projections_georefed__longform'],
 'cli_008_greenhouse_gas_emissions_country_sector': ['cli_008_greenhouse_gas_emissions_country_sector',
  'cli_008_greenhouse_gas_emissions_country_sector_edit',
  'cli_008_greenhouse_gas_emissions_country_sector_georefed__longf'],
 'cli_029_vulnerability_to_cc': ['cli_029_vulnerability_to_cc',
 

In [28]:
all_to_delete = []
for name, tables in matches.items():
    print(name)
    print(tables)
    for table in tables:
        print(table)
        delete = input('Delete this table? type anything for yes, empty for no')
        if delete:
            all_to_delete.append(table)

%20soc_023_fragile_states_index
[]
cit_022_road_traffic_deaths_by_country
['cit_022_road_traffic_deaths_by_country', 'cit_022_road_traffic_deaths_by_country_longform', 'cit_022_road_traffic_deaths_by_country_long', 'cit_022_road_traffic_deaths_by_country_georefed_longform', 'cit_022_road_traffic_deaths_by_country_georefed__longform']
cit_022_road_traffic_deaths_by_country
Delete this table? type anything for yes, empty for no
cit_022_road_traffic_deaths_by_country_longform
Delete this table? type anything for yes, empty for noadf
cit_022_road_traffic_deaths_by_country_long
Delete this table? type anything for yes, empty for noasdf
cit_022_road_traffic_deaths_by_country_georefed_longform
Delete this table? type anything for yes, empty for noasd
cit_022_road_traffic_deaths_by_country_georefed__longform
Delete this table? type anything for yes, empty for no
cit_025_urban_population
['cit_025_urban_population', 'cit_025_urban_population_longform', 'cit_025_urban_population_long', 'cit_025_

In [ ]:
# soc_025_gender_inequality_index

In [31]:
for table in all_to_delete:
    print('dropping table: ' + table)
    dropTable(table)